# Set up

In [13]:
import os
import sys
from datetime import timedelta

import pandas as pd
from feast import FeatureStore
from loguru import logger
from pydantic import BaseModel
from typing import List
from copy import deepcopy

sys.path.insert(0, "..")
from src.id_mapper import IDMapper
from src.utils import generate_negative_samples

# Controller

In [17]:
class Args(BaseModel):
    testing: bool = False
    run_name: str = "000-sequence-modeling"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    
    neg_to_pos_ratio: int = 1

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)

        return self

args = Args().init()

print(args.model_dump_json(indent=2))

{
  "testing": false,
  "run_name": "000-sequence-modeling",
  "notebook_persist_dp": "/mnt/d/projects/recsys/notebooks/data/000-sequence-modeling",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "neg_to_pos_ratio": 1
}


# Test implementation

In [18]:
# Sample input: List of (user_id, item_id) interactions
interactions = [
    (1, 101, 1, 1),
    (1, 102, 2, 2),
    (1, 103, 3, 4),
    (2, 101, 4, 1),
    (2, 104, 5, 2),
    (3, 105, 1, 1),
    (3, 106, 2, 5),
    # Add more interactions as needed
]

# Convert the list to a DataFrame for easier manipulation
df = pd.DataFrame(
    interactions,
    columns=["user_indice", "item_indice", args.rating_col, args.timestamp_col],
)

df

,user_indice,item_indice,rating,timestamp
0,1,101,1,1
1,1,102,2,2
2,1,103,3,4
3,2,101,4,1
4,2,104,5,2
5,3,105,1,1
6,3,106,2,5


In [19]:
neg_df = generate_negative_samples(df, neg_to_pos_ratio=5)

  0%|          | 0/7 [00:00<?, ?it/s]

In [20]:
neg_df

,user_indice,item_indice,rating,timestamp
0,1,105,0,1
0,1,104,0,1
0,1,106,0,1
1,1,105,0,2
1,1,104,0,2
1,1,106,0,2
2,1,104,0,4
2,1,106,0,4
2,1,105,0,4
3,2,103,0,1


# Load data

In [27]:
train_df = pd.read_parquet("../data/train_features.parquet")
val_df = pd.read_parquet("../data/val_features.parquet")
idm = IDMapper().load("../data/idm.json")

In [28]:
assert val_df[args.timestamp_col].min() > train_df[args.timestamp_col].max()
val_timestamp = train_df[args.timestamp_col].max() + timedelta(seconds=1)
logger.info(f"{val_timestamp=}")

2025-11-11 16:13:55.708 | INFO     | __main__:<module>:3 - val_timestamp=Timestamp('2021-08-10 21:28:25.764000')


In [29]:
full_df = pd.concat([train_df, val_df], axis=0)
full_df

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
0,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0050SVNZ8,4.0,2011-11-21 14:27:44.000,1321885664,1,4.000000,1,4.000000,1,...,None,None,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",Video Games,Amazon Basics Carrying Case for Nintendo - New...,[],"[Video Games, Legacy Systems, Nintendo Systems...",None
1,AHATA6X6MYTC3VNBFJ3WIYVK257A,B00LZVNWIA,4.0,2014-08-17 00:00:06.000,1408233606,2,3.500000,2,3.500000,2,...,B0050SVNZ8,1321885664,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1820]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1321885664]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 6]",Computers,Logitech G402 Hyperion Fury FPS Gaming Mouse,[Logitech G402 Hyperion Fury FPS Gaming Mouse],"[Video Games, PC, Accessories, Gaming Mice]",None
2,AHATA6X6MYTC3VNBFJ3WIYVK257A,B0BH98D8GL,5.0,2017-11-26 15:02:34.100,1511708554,2,3.500000,1,3.000000,0,...,"B0050SVNZ8,B00LZVNWIA","1321885664,1408233606","[-1, -1, -1, -1, -1, -1, -1, -1, 1820, 2778]","[-1, -1, -1, -1, -1, -1, -1, -1, 1321885664, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, 8, 7]",Computers,Logitech G433 7.1 Wired Gaming Headset with DT...,[Logitech G433 gaming headset is the premium a...,"[Video Games, Xbox One, Accessories, Headsets]",44.99
3,AHATA6X6MYTC3VNBFJ3WIYVK257A,B074RNL1RX,5.0,2017-11-27 03:26:14.174,1511753174,0,NaN,0,NaN,0,...,"B0050SVNZ8,B00LZVNWIA,B0BH98D8GL","1321885664,1408233606,1511708554","[-1, -1, -1, -1, -1, -1, -1, 1820, 2778, 4549]","[-1, -1, -1, -1, -1, -1, -1, 1321885664, 14082...","[-1, -1, -1, -1, -1, -1, -1, 8, 7, 2]",Video Games,Razer Wolverine Ultimate Officially Licensed X...,[Play anywhere with the Razer Wolverine Ultima...,"[Video Games, PC, Accessories, Controllers]",64.98
4,AHATA6X6MYTC3VNBFJ3WIYVK257A,B089QYP649,5.0,2018-07-08 23:33:40.696,1531092820,26,4.153846,7,4.428571,2,...,"B0050SVNZ8,B00LZVNWIA,B0BH98D8GL,B074RNL1RX","1321885664,1408233606,1511708554,1511753174","[-1, -1, -1, -1, -1, -1, 1820, 2778, 4549, 3757]","[-1, -1, -1, -1, -1, -1, 1321885664, 140823360...","[-1, -1, -1, -1, -1, -1, 8, 7, 5, 5]",Video Games,Turtle Beach Stealth 600 Wireless Surround Sou...,[The Turtle Beach Stealth 600 is the latest wi...,"[Video Games, PlayStation 4, Accessories, Head...",168.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,AE3NRCMFIBBA2XVODR47YYNLKRDA,B001EYUQC8,5.0,2021-11-13 09:59:46.634,1636797586,1,3.000000,1,3.000000,0,...,"B000OLXX86,B000B9RI14,B0050SWQ86,B00CTKHXFO,B0...","1342793426,1362929993,1368549700,1392151606,14...","[-1, -1, -1, 543, 417, 1835, 2407, 3123, 1526,...","[-1, -1, -1, 1342793426, 1362929993, 136854970...","[-1, -1, -1, 8, 8, 8, 8, 8, 8, 7]",Video Games,007 Quantum Of Solace - Playstation 3,[James Bond is back to settle the score in Qua...,"[Video Games, Legacy Systems, PlayStation Syst...",44.49
958,AEV5TZDZQEP24PM3SZ7SNV4TR26Q,B01N3ASPNV,5.0,2022-06-17 07:42:54.083,1655451774,10,4.000000,0,NaN,0,...,"B00HRH79H6,B00JM57VDS,B00VILBF0Y,B072C3VM5F","1423572849,1423573230,1424488991,1523051352","[-1, -1, -1, -1, -1, -1, 2614, 2683, 2975, 3691]","[-1, -1, -1, -1, -1, -1, 1423572849, 142357323...","[-1, -1, -1, -1, -1, -1, 8, 8, 8, 7]",All Electronics,amFilm Tempered Glass Screen Protector for Nin...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",8.91
959,AELRDP5MCGSCANM6GWUXAMBN75LQ,B009AGXH64,5.0,2022-06-03 18:23:36.536,1654280616,1,5.000000,0,NaN,0,...,"B07SSZGYNR,B011AH9A16,B001ELJE5Q,B003N63BPE,B0...","1618466381,1618466731,1623954406,1625231840,16...","[-1, -1, -1, -1, -1, 4168, 3101, 777, 1503, 3526]","[-1, -1, -1, -1, -1, 161846638

# Generate negative samples

In [30]:
full_features_df = deepcopy(full_df)

In [31]:
meta_features = ["main_category", "title", "description", "categories", "price"]

item_timestamp_features = [
    "parent_asin_rating_cnt_365d",
    "parent_asin_rating_avg_prev_rating_365d",
    "parent_asin_rating_cnt_90d",
    "parent_asin_rating_avg_prev_rating_90d",
    "parent_asin_rating_cnt_30d",
    "parent_asin_rating_avg_prev_rating_30d",
    "parent_asin_rating_cnt_7d",
    "parent_asin_rating_avg_prev_rating_7d",
]

item_features_df = full_features_df.drop_duplicates(subset=[args.item_col])[
    [args.item_col, "item_indice", *meta_features]
]

In [32]:
features = [
    "item_sequence",
    "user_rating_list_10_recent_asin_timestamp",
    "item_sequence_ts",
    "item_sequence_ts_bucket",
    "user_id",
    "user_rating_cnt_90d",
    "user_rating_avg_prev_rating_90d",
    "user_rating_list_10_recent_asin",
]

neg_df = generate_negative_samples(
    full_features_df,
    "user_indice",
    "item_indice",
    args.rating_col,
    neg_label=0,
    neg_to_pos_ratio=args.neg_to_pos_ratio,
    seed=args.random_seed,
    features=features,
)

neg_df = neg_df.pipe(
    lambda df: pd.merge(
        df, item_features_df, how="left", on="item_indice", validate="m:1"
    )
)

  0%|          | 0/165260 [00:00<?, ?it/s]

# Get item timestamp feature  for negative samples
The newly generated negative samples would have the new (timestamp, item_indice). To be able to get the item timestamp features like item popularity we need to call feature store

In [33]:
neg_df

,user_indice,item_indice,rating,timestamp,item_sequence,user_rating_list_10_recent_asin_timestamp,item_sequence_ts,item_sequence_ts_bucket,user_id,user_rating_cnt_90d,user_rating_avg_prev_rating_90d,user_rating_list_10_recent_asin,parent_asin,main_category,title,description,categories,price
0,15688,1441,0,2011-11-21 14:27:44.000,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",None,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",AHATA6X6MYTC3VNBFJ3WIYVK257A,1,NaN,None,B003A0T6YS,Video Games,PlayStation 3 PS3 Controler Charge Cable USB A...,[],[],None
1,15688,371,0,2014-08-17 00:00:06.000,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1820]",1321885664,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1321885664]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 6]",AHATA6X6MYTC3VNBFJ3WIYVK257A,1,NaN,B0050SVNZ8,B000935256,Video Games,Soulcalibur 3 - PlayStation 2,"[Product Description, The legendary battle of ...","[Video Games, Legacy Systems, PlayStation Syst...",69.99
2,15688,3068,0,2017-11-26 15:02:34.100,"[-1, -1, -1, -1, -1, -1, -1, -1, 1820, 2778]","1321885664,1408233606","[-1, -1, -1, -1, -1, -1, -1, -1, 1321885664, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, 8, 7]",AHATA6X6MYTC3VNBFJ3WIYVK257A,1,NaN,"B0050SVNZ8,B00LZVNWIA",B00ZHQ39F0,Video Games,Just Dance 2016 - PlayStation 4,[Introducing Just Dance 2016 – the newest game...,"[Video Games, PlayStation 4, Games]",28.0
3,15688,349,0,2017-11-27 03:26:14.174,"[-1, -1, -1, -1, -1, -1, -1, 1820, 2778, 4549]","1321885664,1408233606,1511708554","[-1, -1, -1, -1, -1, -1, -1, 1321885664, 14082...","[-1, -1, -1, -1, -1, -1, -1, 8, 7, 2]",AHATA6X6MYTC3VNBFJ3WIYVK257A,2,5.0,"B0050SVNZ8,B00LZVNWIA,B0BH98D8GL",B00068O278,Video Games,Kingdom Hearts Chain of Memories,[Kingdom Hearts: Chain Of Memories delivers an...,"[Video Games, Legacy Systems, Nintendo Systems...",38.56
4,15688,773,0,2018-07-08 23:33:40.696,"[-1, -1, -1, -1, -1, -1, 1820, 2778, 4549, 3757]","1321885664,1408233606,1511708554,1511753174","[-1, -1, -1, -1, -1, -1, 1321885664, 140823360...","[-1, -1, -1, -1, -1, -1, 8, 7, 5, 5]",AHATA6X6MYTC3VNBFJ3WIYVK257A,1,NaN,"B0050SVNZ8,B00LZVNWIA,B0BH98D8GL,B074RNL1RX",B001ELJE2E,Video Games,GRID - Xbox 360,[GRID is all about 40 of the most incredible r...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",39.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165255,268,511,0,2021-11-13 09:59:46.634,"[-1, -1, -1, 543, 417, 1835, 2407, 3123, 1526,...","1342793426,1362929993,1368549700,1392151606,14...","[-1, -1, -1, 1342793426, 1362929993, 136854970...","[-1, -1, -1, 8, 8, 8, 8, 8, 8, 7]",AE3NRCMFIBBA2XVODR47YYNLKRDA,1,NaN,"B000OLXX86,B000B9RI14,B0050SWQ86,B00CTKHXFO,B0...",B000K9OR4Q,Video Games,Lair - Playstation 3,[In a world ravaged by endless conflict and na...,"[Video Games, Legacy Systems, PlayStation Syst...",24.99
165256,4213,2321,0,2022-06-17 07:42:54.083,"[-1, -1, -1, -1, -1, -1, 2614, 2683, 2975, 3691]","1423572849,1423573230,1424488991,1523051352","[-1, -1, -1, -1, -1, -1, 1423572849, 142357323...","[-1, -1, -1, -1, -1, -1, 8, 8, 8, 7]",AEV5TZDZQEP24PM3SZ7SNV4TR26Q,1,NaN,"B00HRH79H6,B00JM57VDS,B00VILBF0Y,B072C3VM5F",B00B7UHWB2,Video Games,Resident Evil: Revelations - Xbox 360,[The fear that was originally brought to playe...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",52.99
165257,2741,2472,0,2022-06-03 18:23:36.536,"[-1, -1, -1, -1, -1, 4168, 3101, 777, 1503, 3526]","1618466381,1618466731,1623954406,1625231840,16...","[-1, -1, -1, -1, -1, 1618466381, 1618466731, 1...","[-1, -1, -1, -1, -1, 6, 6, 5, 5, 5]",AELRDP5MCGSCANM6GWUXAMBN75LQ,1,NaN,"B07SSZGYNR,B011AH9A16,B001ELJE5Q,B003N63BPE,B0...",B00E1A1SP6,Video Games,Nintendo 3DS XL - Black [Old Model],"[Nintendo 3DS XL, The Nintendo 3DS XL system c...","[Video Games, Legacy Systems, Nintendo Systems...",298.94
165258,16297,2126,0,2021-11-27 00:36:11.015,"[1029, 1035, 220, 1470, 1019, 260, 273, 289, 1...","1595304483,1595304825,1595304936,1606090839,16...","[1595304483, 1595304825, 159530

In [34]:
store = FeatureStore(
    repo_path="..", fs_yaml_file="../feature_store_offline_server.yaml"
)

In [35]:
%%time
ts_features = [f"parent_asin_rating_stats:{feature}" for feature in item_timestamp_features]

neg_ts_features_df = store.get_historical_features(neg_df[[args.item_col, args.timestamp_col]].drop_duplicates(), ts_features).to_df()
assert neg_ts_features_df.duplicated().sum() == 0, display(neg_ts_features_df.loc[neg_ts_features_df.duplicated()])

Using timestamp as the event timestamp. To specify a column explicitly, please name it event_timestamp.
CPU times: user 1.57 s, sys: 201 ms, total: 1.77 s
Wall time: 4min 4s


In [36]:
neg_df = pd.merge(
    neg_df, neg_ts_features_df, on=[args.item_col, args.timestamp_col], how="left"
)
neg_df

,user_indice,item_indice,rating,timestamp,item_sequence,user_rating_list_10_recent_asin_timestamp,item_sequence_ts,item_sequence_ts_bucket,user_id,user_rating_cnt_90d,...,categories,price,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,parent_asin_rating_avg_prev_rating_30d,parent_asin_rating_cnt_7d,parent_asin_rating_avg_prev_rating_7d
0,15688,1441,0,2011-11-21 14:27:44.000,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",None,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",AHATA6X6MYTC3VNBFJ3WIYVK257A,1,...,[],None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15688,371,0,2014-08-17 00:00:06.000,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1820]",1321885664,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1321885664]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 6]",AHATA6X6MYTC3VNBFJ3WIYVK257A,1,...,"[Video Games, Legacy Systems, PlayStation Syst...",69.99,1.0,5.000000,0.0,NaN,0.0,NaN,0.0,NaN
2,15688,3068,0,2017-11-26 15:02:34.100,"[-1, -1, -1, -1, -1, -1, -1, -1, 1820, 2778]","1321885664,1408233606","[-1, -1, -1, -1, -1, -1, -1, -1, 1321885664, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, 8, 7]",AHATA6X6MYTC3VNBFJ3WIYVK257A,1,...,"[Video Games, PlayStation 4, Games]",28.0,26.0,4.461538,4.0,5.0,2.0,5.0,1.0,5.0
3,15688,349,0,2017-11-27 03:26:14.174,"[-1, -1, -1, -1, -1, -1, -1, 1820, 2778, 4549]","1321885664,1408233606,1511708554","[-1, -1, -1, -1, -1, -1, -1, 1321885664, 14082...","[-1, -1, -1, -1, -1, -1, -1, 8, 7, 2]",AHATA6X6MYTC3VNBFJ3WIYVK257A,2,...,"[Video Games, Legacy Systems, Nintendo Systems...",38.56,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
4,15688,773,0,2018-07-08 23:33:40.696,"[-1, -1, -1, -1, -1, -1, 1820, 2778, 4549, 3757]","1321885664,1408233606,1511708554,1511753174","[-1, -1, -1, -1, -1, -1, 1321885664, 140823360...","[-1, -1, -1, -1, -1, -1, 8, 7, 5, 5]",AHATA6X6MYTC3VNBFJ3WIYVK257A,1,...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",39.97,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165255,268,511,0,2021-11-13 09:59:46.634,"[-1, -1, -1, 543, 417, 1835, 2407, 3123, 1526,...","1342793426,1362929993,1368549700,1392151606,14...","[-1, -1, -1, 1342793426, 1362929993, 136854970...","[-1, -1, -1, 8, 8, 8, 8, 8, 8, 7]",AE3NRCMFIBBA2XVODR47YYNLKRDA,1,...,"[Video Games, Legacy Systems, PlayStation Syst...",24.99,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
165256,4213,2321,0,2022-06-17 07:42:54.083,"[-1, -1, -1, -1, -1, -1, 2614, 2683, 2975, 3691]","1423572849,1423573230,1424488991,1523051352","[-1, -1, -1, -1, -1, -1, 1423572849, 142357323...","[-1, -1, -1, -1, -1, -1, 8, 8, 8, 7]",AEV5TZDZQEP24PM3SZ7SNV4TR26Q,1,...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",52.99,4.0,4.750000,1.0,5.0,1.0,5.0,0.0,NaN
165257,2741,2472,0,2022-06-03 18:23:36.536,"[-1, -1, -1, -1, -1, 4168, 3101, 777, 1503, 3526]","1618466381,1618466731,1623954406,1625231840,16...","[-1, -1, -1, -1, -1, 1618466381, 1618466731, 1...","[-1, -1, -1, -1, -1, 6, 6, 5, 5, 5]",AELRDP5MCGSCANM6GWUXAMBN75LQ,1,...,"[Video Games, Legacy Systems, Nintendo Systems...",298.94,1.0,5.000000,0.0,NaN,0.0,NaN,0.0,NaN
165258,16297,2126,0,2021-11-27 00:36:11.015,"[1029, 1035, 220, 1470, 1019, 260, 273, 289, 1...","1595304483,1595304825,1595304936,1606090839,16...","[1595304483, 1595304825, 1595304936, 160609083...","[6, 6, 6, 6, 5, 5, 5, 5, 5, 5]",AHERXKLMQLGPQLW4ZLKD4IRLMZAA,1,...,"[Video Games, Legacy Systems, Xbox Systems]",32.95,1.0,5.000000,1.0,5.0,0.0,NaN,0.0,NaN


# Concating positive data with negative samples

In [37]:
full_features_df = (
    pd.concat([full_features_df, neg_df], axis=0)
    .reset_index(drop=True)
    .sample(frac=1, replace=False, random_state=args.random_seed)
)

In [38]:
key_cols = [
    args.user_col,
    args.item_col,
    "user_indice",
    "item_indice",
    "item_sequence",
    "item_sequence_ts_bucket",
    args.rating_col,
    args.timestamp_col,
]
assert (
    full_features_df[key_cols].isna().sum().sum() == 0
), "Null values found at key colums"

In [39]:
val_timestamp

Timestamp('2021-08-10 21:28:25.764000')

# Split back train test

In [40]:
train_neg_df = full_features_df.loc[lambda df: df[args.timestamp_col].lt(val_timestamp)]
val_neg_df = full_features_df.loc[lambda df: df[args.timestamp_col].ge(val_timestamp)]

In [41]:
train_neg_df

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
251552,AEGX2IOMXXZWDP5LJJVPFARC2LXA,B0014CN2H6,0.0,2015-02-13 16:41:46.000,NaN,1.0,5.000000,0.0,NaN,0.0,...,"B001QCWSLU,B07WWJDWY3,B0077LRPSA,B000U34SZA,B0...","1353957492,1353957582,1353957737,1353957797,13...","[1143, 4229, 2045, 596, 705, 2616, 2232, 2169,...","[1353957492, 1353957582, 1353957737, 135395779...","[6, 6, 6, 6, 6, 6, 6, 6, 0, 0]",Video Games,Shin Megami Tensei: Persona 3 FES - PlayStation 2,[A brand new chapter of Persona 3 featuring 30...,"[Video Games, Legacy Systems, PlayStation Syst...",37.99
325041,AEEEFJCFELMIUQM7S4MN3OJXBE6Q,B0BM5YPLKM,0.0,2008-12-21 01:53:58.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B001EYUXXU,B09MFBJJQJ,B001EYUXXK,B00004S9AF,B0...","1182172867,1211165279,1211422243,1221445917,12...","[-1, -1, -1, -1, -1, 1032, 4491, 1031, 85, 554]","[-1, -1, -1, -1, -1, 1182172867, 1211165279, 1...","[-1, -1, -1, -1, -1, 6, 5, 5, 5, 5]",Computers,"Classic N64 Controller, SAFFUN N64 Wired USB P...",[Customers Frequently Asked Questions: Q: 1. W...,"[Video Games, PC, Accessories, Controllers, Jo...",17.99
293086,AEKGUEYKOTY2NDAZNTRIHXCQBVXQ,B009DL2TBA,0.0,2016-01-30 18:08:13.000,NaN,29.0,4.586207,5.0,4.4,1.0,...,"B005ZBNUUG,B093GLPLLB","1404320532,1444061695","[-1, -1, -1, -1, -1, -1, -1, -1, 1965, 4452]","[-1, -1, -1, -1, -1, -1, -1, -1, 1404320532, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, 6, 5]",Video Games,PlayStation 3 500 GB System,"[Play, Stream, and Watch your favorite games, ...","[Video Games, Legacy Systems, PlayStation Syst...",233.98
52027,AFJO7KXBEW73L2UN7MMKPVHLWORQ,B00006LELI,4.0,2003-11-04 00:26:02.000,1.067906e+09,12.0,4.500000,1.0,4.0,0.0,...,None,None,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",Video Games,Resident Evil Zero - Gamecube,"[Product Description, See how it all began. Th...","[Video Games, Legacy Systems, Nintendo Systems...",29.98
48913,AEZC3FNIGVAQMZT7T2MNXZZOPQZQ,B002BSA2LQ,5.0,2021-05-31 01:24:23.280,1.622424e+09,0.0,NaN,0.0,NaN,0.0,...,"B000UODATY,B001ELJFPA,B07VLCRZ21,B00308JV6I,B0...","1622423201,1622423973,1622423990,1622424022,16...","[604, 808, 4207, 1405, 1583, 1018, 1489, 1446,...","[1622423201, 1622423973, 1622423990, 162242402...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Video Games,Forza Motorsport 3 - Xbox 360,"[Product Description, Whether it’s an exotic s...","[Video Games, Legacy Systems, Xbox Systems, Xb...",31.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250960,AG5K6I36DZU2YYIZOU7F3PNPNQOQ,B07BNKWG2D,0.0,2015-12-27 19:35:23.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B003O6FV8S,B004XV6ST4,B00ZHQ39F0,B001G60JHQ","1297130057,1369331267,1451244466,1451244811","[-1, -1, -1, -1, -1, -1, 1518, 1794, 3068, 1067]","[-1, -1, -1, -1, -1, -1, 1297130057, 136933126...","[-1, -1, -1, -1, -1, -1, 7, 6, 0, 0]",Video Games,LEGO Disney Pixar's The Incredibles - Nintendo...,[Experience the thrilling adventures of the Pa...,"[Video Games, Nintendo Switch, Games]",None
217058,AFCSNRZGIHCXTCTADIEW7SY5BRPQ,B076ZPW7R2,0.0,2013-07-25 18:58:01.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B0030HM2KQ,B0080CAOYM,B00COAGPG2","1361732871,1368381197,1372209466","[-1, -1, -1, -1, -1, -1, -1, 1409, 2102, 2401]","[-1, -1, -1, -1, -1, -1, -1, 1361732871, 13683...","[-1, -1, -1, -1, -1, -1, -1, 5, 5, 4]",Video Games,Senran Kagura Estival Versus - PlayStation 4,"[The high-flying, high-bouncing ninja girls of...","[Video Games, PlayStation 4, Games]",119.0
323468,AGSDWRVEL7YOJTHZLFKW6IIEPE3Q,B00BCX2AZW,0.0,2009-01-30 04:16:53.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B00008URUA,B00005NN5G,B0001VNNE8,B001EYUWBI,B0...","116

In [42]:
val_neg_df

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
165059,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B0B5VZ3358,3.0,2022-01-28 02:06:05.345,1.643336e+09,8.0,3.500000,0.0,NaN,0.0,...,"B00004XOWT,B00005LOXE,B00000JRSB,B011ALES6E,B0...","1496369526,1496369581,1496369625,1496369716,15...","[-1, -1, -1, 114, 144, 26, 3102, 4032, 4209, 4...","[-1, -1, -1, 1496369526, 1496369581, 149636962...","[-1, -1, -1, 7, 7, 7, 7, 6, 6, 6]",Computers,Logitech G604 LIGHTSPEED Gaming Mouse with 15 ...,"[With 15 programmable controls, LIGHTSPEED wir...","[Video Games, PC, Accessories, Gaming Mice]",59.99
330244,AFJCNTMX6SHE6MGI4QSZR35MNSHQ,B00XR3YBM0,0.0,2021-11-25 23:20:07.813,NaN,1.0,1.000000,1.0,1.000000,0.0,...,"B0009VXAM0,B00CMQTVUA,B00D5SZ04K,B00CJ9OTNE,B0...","1193069046,1387405730,1394377448,1394377494,13...","[382, 2400, 2418, 2391, 2552, 4508, 3876, 4455...","[1193069046, 1387405730, 1394377448, 139437749...","[9, 8, 8, 8, 8, 8, 8, 5, 5, 5]",Video Games,WWE 2K16 - PlayStation 4,"[WWE 2K16, Get in the ring and Raise Some Hell...","[Video Games, PlayStation 4, Games]",63.45
164871,AFQPVFAE6XX4B2HICHSV4BOA2DGA,B000N5Z2L4,5.0,2022-03-15 16:31:40.832,1.647362e+09,8.0,4.250000,2.0,5.000000,0.0,...,"B00KSQHX1K,B06XHLM4DX,B01N3ASPNV,B087NNPYP3,B0...","1594828088,1594828163,1594828245,1594828260,15...","[2721, 3573, 3527, 4342, 2793, 4391, 4078, 382...","[1594828088, 1594828163, 1594828245, 159482826...","[6, 6, 6, 6, 6, 6, 0, 0, 0, 0]",Video Games,Xbox Live Gold: 1 Month Membership [Digital Code],"[""Experience the rush of competitive and coope...","[Video Games, Xbox Digital Content, Subscripti...",9.99
330291,AFWMPFOSPMWMIJUMWAPCWBTTAHJQ,B00002SVWG,0.0,2021-12-02 02:13:38.140,NaN,1.0,5.000000,0.0,NaN,0.0,...,"B07HHVF2XG,B07YBXFF5C,B00MAM0ALA,B01MTJA6EV,B0...","1551583563,1551583748,1551583835,1551988017,15...","[4003, 4284, 2791, 3504, 2804, 1875, 3954, 416...","[1551583563, 1551583748, 1551583835, 155198801...","[6, 6, 6, 6, 6, 6, 6, 6, 5, 4]",Video Games,Final Fantasy Tactics,[In Final Fantasy Tactics you must guide your ...,"[Video Games, Legacy Systems, PlayStation Syst...",59.94
329619,AH2FY6PMPRUFGY4CLK4ER356Q2JQ,B008GEH8LQ,0.0,2021-08-14 21:34:28.025,NaN,0.0,NaN,0.0,NaN,0.0,...,"B00ZO1SUSE,B01CKGI0XA,B007BGUGVO,B08NYV2VLS,B0...","1451833963,1466268353,1468513788,1504289735,15...","[-1, 3079, 3294, 2052, 4421, 3650, 3555, 4372,...","[-1, 1451833963, 1466268353, 1468513788, 15042...","[-1, 8, 8, 8, 7, 7, 6, 6, 6, 6]",Video Games,Nintendo 3DS XL - Red/Black,"[Nintendo 3DS XL, The Nintendo 3DS XL system c...","[Video Games, Legacy Systems, Nintendo Systems...",299.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330199,AFZYNSG4NLHKYPWS5N5HDS2WZHXQ,B004QDV76S,0.0,2021-08-22 02:58:42.727,NaN,6.0,3.500000,2.0,4.000000,0.0,...,"B0053BCP40,B003ZDOFF0,B00BN5T30E,B08D3XL1KF,B0...","1417792940,1417793199,1417793211,1621190088,16...","[-1, -1, -1, -1, -1, 1876, 1598, 2352, 4368, 4...","[-1, -1, -1, -1, -1, 1417792940, 1417793199, 1...","[-1, -1, -1, -1, -1, 8, 8, 8, 5, 4]",Video Games,Xbox 360 Black Headset,[The Xbox 360 Headset heightens the experience...,[],5.89
164950,AHXX4FCJWR5WQ4S32FO3P2ATULHA,B07DGMB3L1,5.0,2022-03-21 12:25:12.738,1.647866e+09,2.0,3.500000,0.0,NaN,0.0,...,"B00W9DHUBS,B07YBXFF5C,B00JS9XVNM,B00Z9THEAQ,B0...","1449323768,1463363880,1464722720,1473983851,14...","[2987, 4284, 2684, 3039, 4461, 4367, 3479, 428...","[1449323768, 1463363880, 1464722720, 147398385...","[8, 8, 8, 8, 7, 7, 7, 7, 7, 6]",Video Games,Team Sonic Racing - Nintendo Switch,[Team Sonic Racing combines the best elements ...,"[Video Games, Nintendo Switch, Games]",33.0
164643,AGOPDRGW3FENU7J537W63Z

# Checks

In [43]:
user = val_neg_df.sample(n=1)[args.user_col].values[0]
logger.info(f"Checking user {user}...")
check_df = train_neg_df.loc[lambda df: df[args.user_col].eq(user)].sort_values(
    args.timestamp_col
)
assert (
    check_df[args.rating_col].gt(0).sum() * (args.neg_to_pos_ratio + 1)
    == check_df.shape[0]
), "Unexpected number of pos and neg samples"

2025-11-11 16:44:18.122 | INFO     | __main__:<module>:2 - Checking user AH7AXLRPLDZ2OWOHC6MXIWLGIPGA...


In [44]:
val_check_df = val_neg_df.loc[lambda df: df[args.user_col].eq(user)].sort_values(
    args.timestamp_col
)
item = val_check_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(f"Checking item {item}...")
assert (
    train_neg_df.loc[lambda df: df[args.item_col].eq(item)].shape[0] > 5
), f"Item {item} does not appear much in training data"

2025-11-11 16:44:18.203 | INFO     | __main__:<module>:5 - Checking item B000QJ1NS8...


In [45]:
check_df

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
298312,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B091PJJG69,0.0,2011-01-26 21:29:51,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",Video Games,Wreckfest - Nintendo Switch,[Wreckfest is jam-packed with upgrade and cust...,"[Video Games, Nintendo Switch, Games]",26.29
133052,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B00000K2R4,5.0,2011-01-26 21:29:51,1.296077e+09,1.0,4.000000,1.0,4.000000,0.0,...,None,None,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",Video Games,Sega Dreamcast Console,"[Amazon.com, The Dreamcast system is, in a wor...","[Video Games, Legacy Systems, Sega Systems, Se...",154.99
133053,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B00004SA3J,5.0,2011-02-21 21:34:51,1.298324e+09,0.0,NaN,0.0,NaN,0.0,...,B00000K2R4,1296077391,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 29]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1296077391]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 4]",Video Games,Visual Memory Unit - Blue,[Hyperkin Visual Memory Unit - Blue for Sega D...,"[Video Games, Legacy Systems, Sega Systems, Se...",59.95
298313,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B003EZ1KGQ,0.0,2011-02-21 21:34:51,NaN,NaN,NaN,NaN,NaN,NaN,...,B00000K2R4,1296077391,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 29]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1296077391]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, 4]",Video Games,Spec Ops: The Line - PC,"[Product Description, Spec Ops: The Line is th...","[Video Games, PC, Games]",14.69
133054,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B0050VIOJS,4.0,2011-05-26 20:30:07,1.306442e+09,3.0,5.000000,3.0,5.000000,3.0,...,"B00000K2R4,B00004SA3J","1296077391,1298324091","[-1, -1, -1, -1, -1, -1, -1, -1, 29, 86]","[-1, -1, -1, -1, -1, -1, -1, -1, 1296077391, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, 5, 5]",Video Games,Wii Console with Mario Kart Wii Bundle - White,"[Product Description, Includes: White Wii Cons...","[Video Games, Legacy Systems, Nintendo Systems...",189.98
298314,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B00EP2WNKY,0.0,2011-05-26 20:30:07,NaN,NaN,NaN,NaN,NaN,NaN,...,"B00000K2R4,B00004SA3J","1296077391,1298324091","[-1, -1, -1, -1, -1, -1, -1, -1, 29, 86]","[-1, -1, -1, -1, -1, -1, -1, -1, 1296077391, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, 5, 5]",Video Games,The Sims 4 - Movie Hangout Stuff - Origin PC [...,[Invite your friends over for a movie night an...,"[Video Games, PC, Games]",9.99
133055,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B0039LME9W,4.0,2011-07-18 20:30:15,1.311021e+09,3.0,5.000000,0.0,NaN,0.0,...,"B00000K2R4,B00004SA3J,B0050VIOJS","1296077391,1298324091,1306441807","[-1, -1, -1, -1, -1, -1, -1, 29, 86, 1860]","[-1, -1, -1, -1, -1, -1, -1, 1296077391, 12983...","[-1, -1, -1, -1, -1, -1, -1, 5, 5, 5]",Video Games,Nintendo DSi XL Red Bundle with Mario Kart,"[Product Description, The Nintendo DSi XL is a...","[Video Games, Legacy Systems, Nintendo Systems...",188.73
298315,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B01LPZM7VI,0.0,2011-07-18 20:30:15,NaN,NaN,NaN,NaN,NaN,NaN,...,"B00000K2R4,B00004SA3J,B0050VIOJS","1296077391,1298324091,1306441807","[-1, -1, -1, -1, -1, -1, -1, 29, 86, 1860]","[-1, -1, -1, -1, -1, -1, -1, 1296077391, 12983...","[-1, -1, -1, -1, -1, -1, -1, 5, 5, 5]",Video Games,Xbox Wireless Controller – Black,[Experience the enhanced comfort and feel of t...,"[Video Games, Xbox One, Accessories, Controlle...",59.99
298316,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B01CINIAKS,0.0,2011-09-16 18:22:05,NaN,NaN,NaN,NaN,NaN,NaN,...,"B00000K2R4,B00004SA3J,B0050VIOJS,B0039LME9W","1296077391,1298324091,1306441807,1311021015","[

In [46]:
val_check_df

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
165194,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B000QJ1NS8,3.0,2021-11-14 16:41:02.260,1.636908e+09,0.0,NaN,0.0,NaN,0.0,...,"B0050VIOJS,B0039LME9W,B001ELJDU2,B0050SVMYA,B0...","1306441807,1311021015,1316197325,1326492023,13...","[1860, 1436, 761, 1817, 763, 2121, 1854, 1046,...","[1306441807, 1311021015, 1316197325, 132649202...","[9, 9, 9, 8, 8, 8, 8, 8, 8, 8]",Video Games,Game Boy Advance SP Replacement Battery Pack f...,[Replacement battery for GBA SP],"[Video Games, Legacy Systems, Nintendo Systems...",10.99
330454,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B001MW91IW,0.0,2021-11-14 16:41:02.260,NaN,0.0,NaN,0.0,NaN,0.0,...,"B0050VIOJS,B0039LME9W,B001ELJDU2,B0050SVMYA,B0...","1306441807,1311021015,1316197325,1326492023,13...","[1860, 1436, 761, 1817, 763, 2121, 1854, 1046,...","[1306441807, 1311021015, 1316197325, 132649202...","[9, 9, 9, 8, 8, 8, 8, 8, 8, 8]",None,hossen New 64MB 64 MB Memory Save Card for Pla...,[This card is compatible all versions of the S...,"[Video Games, Legacy Systems, PlayStation Syst...",None
330455,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B078YQ866V,0.0,2022-04-05 23:22:29.450,NaN,0.0,NaN,0.0,NaN,0.0,...,"B0039LME9W,B001ELJDU2,B0050SVMYA,B001ELJDUM,B0...","1311021015,1316197325,1326492023,1330310281,13...","[1436, 761, 1817, 763, 2121, 1854, 1046, 2214,...","[1311021015, 1316197325, 1326492023, 133031028...","[9, 9, 9, 9, 8, 8, 8, 8, 8, 5]",Video Games,Detective Pikachu - Nintendo 3DS,[Crack the case in this new detective adventur...,"[Video Games, Legacy Systems, Nintendo Systems...",45.2
165195,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B0C1K1R6HK,4.0,2022-04-05 23:22:29.450,1.649201e+09,6.0,3.166667,2.0,4.0,0.0,...,"B0039LME9W,B001ELJDU2,B0050SVMYA,B001ELJDUM,B0...","1311021015,1316197325,1326492023,1330310281,13...","[1436, 761, 1817, 763, 2121, 1854, 1046, 2214,...","[1311021015, 1316197325, 1326492023, 133031028...","[9, 9, 9, 9, 8, 8, 8, 8, 8, 5]",Video Games,Xbox Series X,"[Xbox Series X, the fastest, most powerful Xbo...","[Video Games, Legacy Systems, Xbox Systems, Xb...",499.99
165196,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B0BN5DC36N,5.0,2022-05-09 19:38:45.144,1.652125e+09,7.0,4.714286,0.0,NaN,0.0,...,"B001ELJDU2,B0050SVMYA,B001ELJDUM,B0088MVPFQ,B0...","1316197325,1326492023,1330310281,1347656404,13...","[761, 1817, 763, 2121, 1854, 1046, 2214, 2474,...","[1316197325, 1326492023, 1330310281, 134765640...","[9, 9, 9, 8, 8, 8, 8, 8, 5, 5]",Computers,Seagate Horizon Forbidden West Limited Edition...,[Discover new worlds with the officially-licen...,"[Video Games, Legacy Systems, PlayStation Syst...",89.99
330456,AH7AXLRPLDZ2OWOHC6MXIWLGIPGA,B092CGM1ZM,0.0,2022-05-09 19:38:45.144,NaN,0.0,NaN,0.0,NaN,0.0,...,"B001ELJDU2,B0050SVMYA,B001ELJDUM,B0088MVPFQ,B0...","1316197325,1326492023,1330310281,1347656404,13...","[761, 1817, 763, 2121, 1854, 1046, 2214, 2474,...","[1316197325, 1326492023, 1330310281, 134765640...","[9, 9, 9, 8, 8, 8, 8, 8, 5, 5]",Computers,Das Keyboard Model S Wired Mechanical Keyboard...,[Experience faster typing than you ever dreame...,"[Video Games, PC, Accessories, Gaming Keyboards]",119.0


# Persist

In [47]:
full_features_df.to_parquet(
    "../data/full_features_neg_sampling_df.parquet", index=False
)
train_neg_df.to_parquet("../data/train_features_neg_df.parquet", index=False)
val_neg_df.to_parquet("../data/val_features_neg_df.parquet", index=False)